In [28]:
import pandas as pd
import numpy as np

# 1. Escreva uma query que retorna a quantidade de linhas na tabela Sales.SalesOrderDetail pelo campo SalesOrderID, desde que tenham pelo menos três linhas de detalhes.
## Acha a quantidade de linhas de detalhes (SalesOrderDetailID) por SalesOrderID.
- Após fazer uma análise exploratória dos dados, foi verificado que **para cada SalesOrderID existe uma SalesOrderDetailID.**
 - Foi feito um agrupamento da coluna SalesOrderID com a função de agregação size() para achar o número de linhas para cada SalesOrderID, **que é o mesmo número de linhas da coluna SalesOrderDetailID.**

In [29]:
def question_1(base1="Sales.SalesOrderDetail.csv"):
    
    # Importa base
    SalesOrderDetail = pd.read_csv(f"../data/{base1}", sep=";")
    
    # Agrupa SalesOrderDetail por SalesOrderID com a função de agregação size()
    grouped_SalesOrderDetail = SalesOrderDetail.groupby('SalesOrderID').size().reset_index(name='Entries')
    
    # Seleciona as linhas com número de registros maior ou igual a 3
    answer1 = grouped_SalesOrderDetail.loc[grouped_SalesOrderDetail["Entries"] >= 3]
    
    # Retorna o tamanho do dataframe answer, que é o resultado esperado
    return len(answer1)

In [30]:
question_1("Sales.SalesOrderDetail.csv")

12757

# 2. Escreva uma query que ligue as tabelas Sales.SalesOrderDetail, Sales.SpecialOfferProduct e Production.Product e retorne os 3 produtos (Name) mais vendidos (pela soma de OrderQty), agrupados pelo número de dias para manufatura (DaysToManufacture).
- Após análise exploratória dos dados, foi observado que a chave comum entre as 3 bases é a coluna **ProductID.**
- As colunas relevantes para alcançar o resultado desejado são:
 - **OrderQty**, encontrada somente na tabela Sales.SalesOrderDetail;
 - **DaysToManufacture**, encontrada nas tabelas Sales.SpecialOfferProduct e Production.Product;
 - **Name**, encontrada nas tabelas Sales.SpecialOfferProduct e Production.Product.
- Como DaysToManufacture e Name estão nas duas tabelas Sales.SpecialOfferProduct e Production.Product, e elas possuem os mesmos produtos, inclusive com o mesmo número de registros (504), não é necessário fazer uma relação entre essas duas tabelas.
- Foi feito um merge entre as tabelas Sales.SalesOrderDetail.csv e Production.Product.csv utilizando como chave o ProductID
 - O resultado foi agrupado por DaysToManufacture e Name com a função de agregação sum na coluna OrderQty.
 - Foram selecionados os 3 produtos mais vendidos.
 
 # ALTERAR DESCRIÇÃO

In [420]:
# def question_2(base1="Sales.SalesOrderDetail.csv", base2="Sales.SpecialOfferProduct.csv",
#                base3="Production.Product.csv"):
    
#     # Importa bases
#     SalesOrderDetail = pd.read_csv(f"../data/{base1}", sep=";")
#     SpecialOfferProduct = pd.read_csv(f"../data/{base3}", sep=";")
#     Product = pd.read_csv(f"../data/{base3}", sep=";")
    
#     # Não é necessário fazer o merge entre SpecialOfferProduct e Product.
#     # Não foi executado, mas, como foi solicitado no enunciado, segue:
#     # SpecialOfferProduct_Product = pd.merge(SpecialOfferProduct, Product, on="ProductID", how="inner")
    
#     # Relaziona Sales.SalesOrderDetail e Production.Product para trazer a coluna OrderQty
#     Product_SalesOrderDetail = pd.merge(Product, SalesOrderDetail[["OrderQty", "ProductID"]], on="ProductID", how="inner")
    
#     # Seleciona as colunas relevantes para diminuir o dataframe
#     Product_SalesOrderDetail = Product_SalesOrderDetail[["ProductID", "Name", "OrderQty", "DaysToManufacture"]]
    
#     # Agrupa o dataframe e aplica sum na coluna OrderQty
#     Product_SalesOrderDetail = Product_SalesOrderDetail.groupby(["DaysToManufacture", "Name"]).agg({'OrderQty':sum,})
    
#     # Seleciona os 3 produtos mais vendidos, retorna o nome agrupado por DaysToManufacture
#     answer2 = Product_SalesOrderDetail.groupby('DaysToManufacture').apply(lambda x: x.nlargest(3,"OrderQty")).reset_index(level=0, drop=True)
    

#     return answer2


def question_2(base1="Sales.SalesOrderDetail.csv", base2="Sales.SpecialOfferProduct.csv",
               base3="Production.Product.csv"):
    
    # Importa bases
    SalesOrderDetail = pd.read_csv(f"../data/{base1}", sep=";")
    SpecialOfferProduct = pd.read_csv(f"../data/{base2}", sep=";")
    Product = pd.read_csv(f"../data/{base3}", sep=";")
    
    # Relaciona SpecialOfferProduct e Product para trazer os produtos a mais de SpecialOfferProduct
    SpecialOfferProduct_Product = pd.merge(SpecialOfferProduct, Product, on="ProductID", how="left")
    
    # Relaciona Sales.SalesOrderDetail e Production.Product para trazer a coluna OrderQty
    Product_SalesOrderDetail = pd.merge(SpecialOfferProduct_Product, SalesOrderDetail[["OrderQty", "ProductID"]], on="ProductID", how="inner")
    
    # Seleciona as colunas relevantes para diminuir o dataframe
    Product_SalesOrderDetail = Product_SalesOrderDetail[["ProductID", "Name", "OrderQty", "DaysToManufacture"]]
    
    # Agrupa o dataframe e aplica sum na coluna OrderQty
    Product_SalesOrderDetail = Product_SalesOrderDetail.groupby(["DaysToManufacture", "Name"]).agg({'OrderQty':sum,})
    
    # Seleciona os 3 produtos mais vendidos, retorna o nome agrupado por DaysToManufacture
    answer2 = Product_SalesOrderDetail.groupby('DaysToManufacture').apply(lambda x: x.nlargest(3,"OrderQty")).reset_index(level=0, drop=True)
    
    return answer2

In [421]:
question_2()

OrderQty
DaysToManufacture Name                                    
0                 Sport-100 Helmet, Blue             33715
                  AWC Logo Cap                       33244
                  Sport-100 Helmet, Black            32660
1                 LL Road Frame - Black, 52           4743
                  ML Road Frame-W - Yellow, 44        4305
                  LL Road Frame - Red, 44             2973
2                 HL Mountain Frame - Silver, 38      2412
                  ML Road Frame-W - Yellow, 38         895
                  LL Mountain Frame - Silver, 40       884
4                 Mountain-200 Black, 38             11908
                  Road-650 Red, 44                    9016
                  Mountain-200 Black, 42              7992

# 3. Escreva uma query ligando as tabelas Person.Person, Sales.Customer e Sales.SalesOrderHeader de forma a obter uma lista de nomes de clientes e uma contagem de pedidos efetuados.
- Nome dos cliente consta na tabela Person.Person
- Após análise exploratória, foi identificado que as chaves BusinessEntityID e CustomerID são as mesmas

In [403]:
def question_3(base1="Person.Person.csv", base2="Sales.Customer.csv",
               base3="Sales.SalesOrderHeader.csv"):
    
    # Importa bases
    Person = pd.read_csv(f"../data/{base1}", sep=";")
    Customer = pd.read_csv(f"../data/{base2}", sep=";")
    SalesOrderHeader = pd.read_csv(f"../data/{base3}", sep=";")
    
    # Nesse caso não é necessário
    Person_Customer = pd.merge(Customer, Person, left_on="CustomerID", right_on="BusinessEntityID", how="inner")
    
    
    SalesOrderHeader_Customer = pd.merge(SalesOrderHeader, Person_Customer[["FirstName", "MiddleName", "LastName", "CustomerID"]],
                               on="CustomerID", how="inner")
    
    SalesOrderHeader_Customer["Name"] = SalesOrderHeader_Customer["FirstName"]+" "+SalesOrderHeader_Customer["MiddleName"]+" "+SalesOrderHeader_Customer["LastName"]
    
    SalesOrderHeader_Customer = SalesOrderHeader_Customer.groupby("Name")["SalesOrderID"].size().sort_values(ascending=False)
    
    answer3 = SalesOrderHeader_Customer.to_frame(name="Number_of_Orders")
    
    return answer3

In [404]:
question_3()

,Number_of_Orders
Name,
Morgan C Miller,28
Grace R Lewis,27
Natalie M Martin,27
Grace J Lee,27
Marshall M Shen,27
...,...
Kaitlyn R Howard,1
Kaitlyn O Butler,1
Kaitlyn M Russell,1


In [382]:
list(question_3())

['BusinessEntityID',
 'PersonType',
 'NameStyle',
 'Title',
 'FirstName',
 'MiddleName',
 'LastName',
 'Suffix',
 'EmailPromotion',
 'AdditionalContactInfo',
 'Demographics',
 'rowguid',
 'ModifiedDate']

In [379]:
list(question_3())

['CustomerID',
 'PersonID',
 'StoreID',
 'TerritoryID',
 'AccountNumber',
 'rowguid',
 'ModifiedDate']

In [377]:
list(question_3())

['SalesOrderID',
 'RevisionNumber',
 'OrderDate',
 'DueDate',
 'ShipDate',
 'Status',
 'OnlineOrderFlag',
 'SalesOrderNumber',
 'PurchaseOrderNumber',
 'AccountNumber',
 'CustomerID',
 'SalesPersonID',
 'TerritoryID',
 'BillToAddressID',
 'ShipToAddressID',
 'ShipMethodID',
 'CreditCardID',
 'CreditCardApprovalCode',
 'CurrencyRateID',
 'SubTotal',
 'TaxAmt',
 'Freight',
 'TotalDue',
 'Comment',
 'rowguid',
 'ModifiedDate']

# 4. Escreva uma query usando as tabelas Sales.SalesOrderHeader, Sales.SalesOrderDetail e Production.Product, de forma a obter a soma total de produtos (OrderQty) por ProductID e OrderDate.

In [430]:
def question_4(base1="Sales.SalesOrderHeader.csv", base2="Sales.SalesOrderDetail.csv",
               base3="Production.Product.csv"):
    
    # Importa bases
    SalesOrderHeader = pd.read_csv(f"../data/{base1}", sep=";")
    SalesOrderDetail = pd.read_csv(f"../data/{base2}", sep=";")
    Product = pd.read_csv(f"../data/{base3}", sep=";")

    
    SalesOrderHeader_Detail = pd.merge(SalesOrderHeader, SalesOrderDetail, on="SalesOrderID", how="inner")
    

    SalesOrderHeader_Detail_Product = pd.merge(SalesOrderHeader_Detail, Product[["ProductID"]], on="ProductID", how="inner")
    
    # Seleciona as colunas relevantes para diminuir o dataframe
    SalesOrderHeader_Detail_Product = SalesOrderHeader_Detail_Product[["ProductID", 
                                                                       "OrderQty", "OrderDate"]]
    
    # Agrupa o dataframe e aplica sum na coluna OrderQty
    answer4 = SalesOrderHeader_Detail_Product.groupby(["ProductID", "OrderDate"]).agg({'OrderQty':sum,})

    
    return answer4

In [431]:
question_4()

OrderQty
ProductID OrderDate                        
707       2011-05-31 00:00:00.000        24
          2011-07-01 00:00:00.000        58
          2011-08-01 00:00:00.000        55
          2011-08-31 00:00:00.000        41
          2011-10-01 00:00:00.000        77
...                                     ...
999       2014-05-25 00:00:00.000         3
          2014-05-27 00:00:00.000         1
          2014-05-28 00:00:00.000         3
          2014-05-29 00:00:00.000         1
          2014-05-30 00:00:00.000         1

[26878 rows x 1 columns]

# 5. Escreva uma query mostrando os campos SalesOrderID, OrderDate e TotalDue da tabela Sales.SalesOrderHeader. Obtenha apenas as linhas onde a ordem tenha sido feita durante o mês de setembro/2011 e o total devido esteja acima de 1.000. Ordene pelo total devido decrescente.

### Análise resultado 3

In [209]:
Person, Customer, SalesOrderHeader = question_3()

In [210]:
len(Person)

19972

In [217]:
# pd.set_option('max_columns', None)
len(Customer)

31465

In [216]:
len(SalesOrderHeader)

31465

In [222]:
len(pd.merge(Customer, SalesOrderHeader, on="SalesOrderID", how="outer"))

31465

In [223]:
len(Customer.merge(SalesOrderHeader, on='SalesOrderID', how='outer', indicator=True)
     .query('_merge != "both"')
     .drop('_merge', 1))

0

In [111]:
# set(Person.columns).intersection(set(Customer.columns))
# set(Person.columns).intersection(set(SalesOrderHeader.columns))
# set(Customer.columns).intersection(set(SalesOrderHeader.columns))

### Análise resultado 2

In [351]:
SpecialOfferProduct = pd.read_csv("../data/Sales.SpecialOfferProduct.csv", sep=";")
Product = pd.read_csv("../data/Production.Product.csv", sep=";")

In [353]:
Product

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,0,0,NaN,1000,750,"0,00","0,00",NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00.000,NaN,NaN,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,0,0,NaN,1000,750,"0,00","0,00",NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00.000,NaN,NaN,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,1,0,NaN,800,600,"0,00","0,00",NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00.000,NaN,NaN,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,0,0,NaN,800,600,"0,00","0,00",NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00.000,NaN,NaN,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,1,0,NaN,800,600,"0,00","0,00",NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00.000,NaN,NaN,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,995,ML Bottom Bracket,BB-8107,1,1,NaN,500,375,"44,9506","101,24",NaN,NaN,G,168.00,1,NaN,M,NaN,5.0,96.0,2013-05-30 00:00:00.000,NaN,NaN,71AB847F-D091-42D6-B735-7B0C2D82FC84,2014-02-08 10:01:36.827
500,996,HL Bottom Bracket,BB-9108,1,1,NaN,500,375,"53,9416","121,49",NaN,NaN,G,170.00,1,NaN,H,NaN,5.0,97.0,2013-05-30 00:00:00.000,NaN,NaN,230C47C5-08B2-4CE3-B706-69C0BDD62965,2014-02-08 10:01:36.827
501,997,"Road-750 Black, 44",BK-R19B-44,1,1,Black,100,75,"343,6496","539,99",44,CM,LB,19.77,4,R,L,U,2.0,31.0,2013-05-30 00:00:00.000,NaN,NaN,44CE4802-409F-43AB-9B27-CA53421805BE,2014-02-08 10:01:36.827
502,998,"Road-750 Black, 48",BK-R19B-48,1,1,Black,100,75,"343,6496","539,99",48,CM,LB,20.13,4,R,L,U,2.0,31.0,2013-05-30 00:00:00.000,NaN,NaN,3DE9A212-1D49-40B6-B10A-F564D981DBDE,2014-02-08 10:01:36.827


In [193]:
SalesOrderDetail.groupby(["ProductID"]).agg({"OrderQty":sum}).sort_values(by=["OrderQty"])

,OrderQty
ProductID,
897,4
942,7
943,8
911,10
927,15
...,...
708,6532
715,6592
711,6743


In [227]:
len(SpecialOfferProduct.merge(Product, on='ProductID', how='outer', indicator=True)
     .query('_merge != "both"')
     .drop('_merge', 1))

209

In [229]:
len(SpecialOfferProduct.merge(Product, on='ProductID', how='right', indicator=True)
     .query('_merge == "right_only"')
     .drop('_merge', 1))

209

In [233]:
len(pd.merge(SpecialOfferProduct, Product, on="ProductID", how="right"))

747